In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import glob

/home/asha/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
file_names=glob.glob("./data/train/*.jpg")
file_names.sort()

In [3]:
len(file_names)

10222

In [4]:
y_train=pd.read_csv("./data/labels.csv",delimiter=",")
y_true=y_train['breed']


In [5]:
def breed_to_numbers(y_train):
    unq=np.unique(y_train)
    n=len(unq)
    numbers = list(range(0, n))
    y_dict=dict(zip(unq,numbers))
    y_true=[]
    for label in y_train:
        n=y_dict[label]
        y_true.append(n)
    return y_true

In [6]:
y_true=breed_to_numbers(y_true)


In [7]:
#divide the data in to training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( file_names, y_true, test_size=0.2)

In [8]:
print len(X_train)
print len(X_test)
print len(y_train)
print len(y_test)

8177
2045
8177
2045


In [9]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [224,224])
    
    return image, label

In [10]:
batch_size=32

In [11]:
label=tf.one_hot(y_train,120)
dataset=tf.data.Dataset.from_tensor_slices((X_train,label))
dataset=dataset.map(parse_function)
dataset=dataset.batch(batch_size)



In [12]:

iterator = dataset.make_initializable_iterator()
def train_data():
    x,y = iterator.get_next()
    return x,y
    


In [13]:
label_test=tf.one_hot(y_test,120)
dataset_test=tf.data.Dataset.from_tensor_slices((X_test,label_test))
dataset_test=dataset_test.map(parse_function)
dataset_test=dataset_test.batch(batch_size)

In [14]:
iterator_test = dataset_test.make_initializable_iterator()
def test_data():  
    x,y = iterator_test.get_next()
    return x,y

In [15]:
a = tf.placeholder(tf.bool)  #placeholder for a single boolean value
x,y= tf.cond(tf.equal(a, tf.constant(True)), lambda:train_data(), lambda:test_data())

In [16]:
 
#convolution, relu and max poling for layer 1
conv1=tf.layers.conv2d(x,32,3,strides=1,activation=tf.nn.relu)  
conv1=tf.layers.max_pooling2d(conv1,2,2,padding='SAME')

conv1=tf.layers.batch_normalization(conv1)

    
#convolution, relu and max poling for layer 2
conv2=tf.layers.conv2d(conv1,64,5,strides=1,activation=tf.nn.relu)  
conv2=tf.layers.max_pooling2d(conv2,2,2,padding='SAME')
conv2=tf.layers.batch_normalization(conv2)


conv3=tf.layers.conv2d(conv2,128,5,strides=1,activation=tf.nn.relu)  
conv3=tf.layers.max_pooling2d(conv3,2,2,padding='SAME')

conv3=tf.layers.batch_normalization(conv3)


fc1=tf.contrib.layers.flatten(conv3)
#fully connected layer
fc1=tf.layers.dense(inputs=fc1, units=2048,activation=tf.nn.relu)
fc2=tf.layers.dense(inputs=fc1, units=512,activation=tf.nn.relu)
logits=tf.layers.dense(inputs=fc2, units=120)
#softmax for probabilities
#prob=tf.nn.softmax(logits)
    
#calculating loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op = optimizer.minimize(loss_op)
    
#calculating accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init=tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [17]:
batches=len(X_train)/batch_size
batches=int(batches)
epochs=4
print batches
batches_test=len(X_test)/batch_size
print batches_test

255
63


In [18]:

with tf.Session() as sess:
    sess.run(init)
    sess.run(iterator_test.initializer)
    
    for epoch in range(epochs):
        sess.run(iterator.initializer)
        print "Epoch:",epoch ,"running"
        
        
        for batch in range(batches):
            if batch%10!=0:
                sess.run(train_op,feed_dict={a:True})
            if batch%10==0:
                loss,acc,_=sess.run([loss_op,accuracy,train_op],feed_dict={a:True})
                print "batch",batch,"loss:",loss,"accuracy",acc
            
                 
                
        print "Epoch:",epoch,"ended"
    print "optimazion finished"
    accuracy_list=[]
    
    
    for batch in range(batches_test):
            acc=sess.run(accuracy,feed_dict={a:False})
            accuracy_list.append(acc)
            if batch%10==0:
                print "batch",batch,"accuracy:",acc
    n=sum(accuracy_list)/len(accuracy_list)
    print "overall Testing Accuracy: ",n
   
   
  
        
        


Epoch: 0 running
batch 0 loss: 4.7949877 accuracy 0.0
batch 10 loss: 4.7445507 accuracy 0.03125
batch 20 loss: 4.778106 accuracy 0.03125
batch 30 loss: 4.7881536 accuracy 0.0
batch 40 loss: 4.7975674 accuracy 0.0625
batch 50 loss: 4.793742 accuracy 0.0
batch 60 loss: 4.7747984 accuracy 0.03125
batch 70 loss: 4.796772 accuracy 0.0
batch 80 loss: 4.770176 accuracy 0.03125
batch 90 loss: 4.782983 accuracy 0.09375
batch 100 loss: 4.7494564 accuracy 0.03125
batch 110 loss: 4.808174 accuracy 0.03125
batch 120 loss: 4.7425528 accuracy 0.0
batch 130 loss: 4.7932897 accuracy 0.03125
batch 140 loss: 4.696476 accuracy 0.09375
batch 150 loss: 4.5091996 accuracy 0.09375
batch 160 loss: 4.650653 accuracy 0.09375
batch 170 loss: 4.707104 accuracy 0.03125
batch 180 loss: 4.6074796 accuracy 0.0625
batch 190 loss: 4.7390623 accuracy 0.0
batch 200 loss: 4.6320896 accuracy 0.03125
batch 210 loss: 4.610182 accuracy 0.0
batch 220 loss: 4.5299673 accuracy 0.0625
batch 230 loss: 4.7855096 accuracy 0.0
batch 2